In [1]:
from torch.utils.data import Dataset
import os
import json
from PIL import Image
from torchvision import transforms, models, datasets
from torchvision.ops import nms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

NameError: name 'torch' is not defined

In [ ]:
IMAGE_ROOT = 'images/images'
DF_RAW = pd.read_csv('df.csv')
print(DF_RAW.head())

In [9]:
import os

class OpenImages(Dataset):
    def __init__(self, image_folder, annotation_folder):
        self.image_folder = image_folder
        self.annotation_folder = annotation_folder
        self.image_files = []

        # Recursively search for image files in the image folder and its subfolders
        for root, _, files in os.walk(image_folder):
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    relative_path = os.path.relpath(root, image_folder)
                    self.image_files.append(os.path.join(relative_path, file))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, ix):
        print(self.image_files)
        image_file = self.image_files[ix]
        image_path = os.path.join(self.image_folder, image_file)
        image = cv2.imread(image_path, 1)[...,::-1]  # convert BGR to RGB
        h, w, _ = image.shape

        # Construct the path for the YOLO annotation file
        annotation_file = os.path.join(self.annotation_folder, 
                                       os.path.splitext(image_file)[0] + '.txt')
        
        boxes = []
        classes = []

        if os.path.exists(annotation_file):
            with open(annotation_file, 'r') as f:
                for line in f:
                    data = line.strip().split()
                    if len(data) == 5:
                        class_id, x_center, y_center, width, height = map(float, data)
                        x_min = int((x_center - width/2) * w)
                        y_min = int((y_center - height/2) * h)
                        x_max = int((x_center + width/2) * w)
                        y_max = int((y_center + height/2) * h)
                        boxes.append([x_min, y_min, x_max, y_max])
                        classes.append(int(class_id))

        return image, boxes, classes, image_path
    
IMAGE_ROOT = 'images'
ANNOTATION_ROOT = 'annotations'

ds = OpenImages(image_folder=IMAGE_ROOT, annotation_folder=ANNOTATION_ROOT)
im, bbs, clss, _ = ds[9]

['laptop\\laptop_0.jpg', 'laptop\\laptop_1.jpg', 'laptop\\laptop_10.jpg', 'laptop\\laptop_100.jpg', 'laptop\\laptop_101.jpg', 'laptop\\laptop_102.jpg', 'laptop\\laptop_103.jpg', 'laptop\\laptop_104.jpg', 'laptop\\laptop_105.jpg', 'laptop\\laptop_106.jpg', 'laptop\\laptop_107.jpg', 'laptop\\laptop_108.jpg', 'laptop\\laptop_11.jpg', 'laptop\\laptop_12.jpg', 'laptop\\laptop_13.jpg', 'laptop\\laptop_14.jpg', 'laptop\\laptop_15.jpg', 'laptop\\laptop_16.jpg', 'laptop\\laptop_17.jpg', 'laptop\\laptop_18.jpg', 'laptop\\laptop_19.jpg', 'laptop\\laptop_2.jpg', 'laptop\\laptop_20.jpg', 'laptop\\laptop_21.jpg', 'laptop\\laptop_22.jpg', 'laptop\\laptop_23.jpg', 'laptop\\laptop_24.jpg', 'laptop\\laptop_25.jpg', 'laptop\\laptop_26.jpg', 'laptop\\laptop_27.jpg', 'laptop\\laptop_28.jpg', 'laptop\\laptop_29.jpg', 'laptop\\laptop_3.jpg', 'laptop\\laptop_30.jpg', 'laptop\\laptop_31.jpg', 'laptop\\laptop_32.jpg', 'laptop\\laptop_33.jpg', 'laptop\\laptop_34.jpg', 'laptop\\laptop_35.jpg', 'laptop\\laptop_36.

NameError: name 'show' is not defined

In [1]:
import matplotlib.pyplot as plt
import cv2

image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

fig, ax = plt.subplots()
ax.imshow(image)

NameError: name 'im' is not defined

In [2]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = CustomImageDataset(img_dir='images', 
                             annot_dir='annotions',
                             transform=transform)

In [ ]:
im, bbs, clss, _ = ds[9]
show(im, bbs=bbs, texts=clss, sz=10)

In [3]:
from torch.utils.data import DataLoader

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)